In [1]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
from datetime import datetime, timedelta
import time

import os
import sys

from google.cloud import bigquery
from google.oauth2 import service_account
import gspread
from gspread_dataframe import set_with_dataframe
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload

import warnings

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"
sheetskey_path = KEY_PATH + "mido-project-426906-41a4b6d0e3db.json"
midopluskey_path = KEY_PATH + "midoplus.json"

warnings.filterwarnings("ignore")

In [3]:
def get_service_key(servicekey_path):

    with open(servicekey_path) as f:
        data = json.load(f)
    return data

In [4]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [5]:
create_bigquery_client(midopluskey_path)

In [6]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [34]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

#### 빅쿼리

In [35]:
shopping_prod_df = get_dataframe_from_bigquery('DATA_MARTS', 'g2b_prod_data', midopluskey_path)
shopping_prod_df_fin = shopping_prod_df.fillna('')

#### 구글 스프레드 시트

In [7]:
# Google Sheets 클라이언트 생성
sheets_scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
                "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

sheets_creds = service_account.Credentials.from_service_account_file(midopluskey_path, scopes=sheets_scope)
gc = gspread.authorize(sheets_creds)

#### 제품규격서

In [256]:
prod_spec_info = pd.read_csv('C:/py_src/midoproject/data/품질기준 및 규격.csv',encoding='cp949')

In [257]:
# 데이터구분
prod_spec_df_ks = prod_spec_info[prod_spec_info['업체명']=='KS인증'].reset_index(drop=True)
prod_spec_df_fin = prod_spec_info[prod_spec_info['업체명']!='KS인증'].reset_index(drop=True)

In [258]:
# 데이터 병합
prod_spec_df_fin['제품']='t' + prod_spec_df_fin['파일길이'] ## 매핑 키 생성
prod_spec_df_fin['제품'] = prod_spec_df_fin['제품'].str.replace(' ','')
merge_prod_spec_df = pd.merge(shopping_prod_df_fin,prod_spec_df_fin,how='left',on=['업체명','제품'])

In [259]:
# 필요컬럼추출
need_col_num1 = merge_prod_spec_df.columns.get_loc('소재지')
need_col_num2 = merge_prod_spec_df.columns.get_loc('에코여부')
need_col = list(merge_prod_spec_df.columns[:need_col_num1]) + list(merge_prod_spec_df.columns[need_col_num2:])

In [260]:
# 최종데이터셋
prod_spec_df_final = merge_prod_spec_df[['쇼핑계약번호','쇼핑계약순번','업체명','계약방법_x','물품명','세부물품명','물품식별명','계약부서명','인증목록','우수조달물품여부','다수공급경쟁자여부','중소기업자간경쟁제품여부',
                                         '금액','납품장소명','인도조건명','공급지역명','납품기한','제품','잔디구조이중여부','충전재여부','충격흡수패드여부','에코여부','구분','모델명','가격','파일길이','파일',
                                         '기포지','백코팅','충전재','패드','원사총섬도','원사단사섬도','단위면적당 파일사 무게','파일사 무게 비율','마모강도(2000회)','방염성능(45°법)_잔염시간',
                                         '방염성능(45°법)_탄화거리','접합강도_y','인발력_상태','인발력_상온 침수 후(23°C, 72 h)','인발력_상온 침수 후(N)','내광성_인발력','내광성_변퇴색','충격 흡수성',
                                         '수직 방향 변형','한계 하강 높이','회전 저항','피부 표면 마찰','공의 반발력','공 구름','투수 성능','스터드마모_충격 흡수성','스터드마모_수직 방향 변형',
                                         '스터드마모_파일 인발력','XL스터드마모_충격 흡수성','XL스터드마모_수직 방향 변형','XL스터드마모_회전 저항','XL스터드마모_공의 반발력','XL스터드마모_공 구름','기타']]

In [266]:
 # 55mm 제품
prod_spec_df_55 = prod_spec_df_final[prod_spec_df_final['제품']=='t55mm'].reset_index(drop=True)

In [267]:
comp_col = ['원사단사섬도','단위면적당 파일사 무게','파일사 무게 비율','접합강도_y','인발력_상태','인발력_상온 침수 후(23°C, 72 h)','마모강도(2000회)','방염성능(45°법)_잔염시간','방염성능(45°법)_탄화거리',
            '충격 흡수성','수직 방향 변형','회전 저항','피부 표면 마찰','공의 반발력','스터드마모_충격 흡수성','스터드마모_수직 방향 변형','스터드마모_파일 인발력','투수 성능']

prod_spec_df_55['원사단사섬도'] = prod_spec_df_55['원사단사섬도'].str.split(' ').str[0]
prod_spec_df_55['단위면적당 파일사 무게'] = prod_spec_df_55['단위면적당 파일사 무게'].str.split(' ').str[0]
prod_spec_df_55['파일사 무게 비율'] = prod_spec_df_55['파일사 무게 비율'].str.split(' ').str[0]
prod_spec_df_55['접합강도_y'] = prod_spec_df_55['접합강도_y'].str.split(' ').str[0]
prod_spec_df_55['인발력_상태'] = prod_spec_df_55['인발력_상태'].str.split(' ').str[0]
prod_spec_df_55['인발력_상온 침수 후(23°C, 72 h)'] = prod_spec_df_55['인발력_상온 침수 후(23°C, 72 h)'].str.split(' ').str[-2]
prod_spec_df_55['마모강도(2000회)'] = prod_spec_df_55['마모강도(2000회)'].str.split(' ').str[0]
prod_spec_df_55['방염성능(45°법)_잔염시간'] = prod_spec_df_55['방염성능(45°법)_잔염시간'].str.split(' ').str[0]
prod_spec_df_55['방염성능(45°법)_탄화거리'] = prod_spec_df_55['방염성능(45°법)_탄화거리'].str.split(' ').str[0]
prod_spec_df_55['충격 흡수성'] = prod_spec_df_55['충격 흡수성'].str.split(' ').str[0]
prod_spec_df_55['수직 방향 변형'] = prod_spec_df_55['수직 방향 변형'].str.split(' ').str[0]
prod_spec_df_55['회전 저항'] = prod_spec_df_55['회전 저항'].str.split(' ').str[0]
prod_spec_df_55['피부 표면 마찰'] = prod_spec_df_55['피부 표면 마찰'].str.split(' ').str[0]
prod_spec_df_55['공의 반발력'] = prod_spec_df_55['공의 반발력'].str.split(' ').str[0]
prod_spec_df_55['스터드마모_충격 흡수성'] = prod_spec_df_55['스터드마모_충격 흡수성'].str.split(' ').str[0]
prod_spec_df_55['스터드마모_수직 방향 변형'] = prod_spec_df_55['스터드마모_수직 방향 변형'].str.split(' ').str[0]
prod_spec_df_55['스터드마모_파일 인발력'] = prod_spec_df_55['스터드마모_파일 인발력'].str.split(' ').str[0]
prod_spec_df_55['투수 성능'] = prod_spec_df_55['투수 성능'].str.split(' ').str[0]

prod_spec_df_55[comp_col] = prod_spec_df_55[comp_col].fillna(0)

prod_spec_df_55['원사단사섬도'] = prod_spec_df_55['원사단사섬도'].astype(float)
prod_spec_df_55['단위면적당 파일사 무게'] = prod_spec_df_55['단위면적당 파일사 무게'].astype(float)
prod_spec_df_55['파일사 무게 비율'] = prod_spec_df_55['파일사 무게 비율'].astype(float)
prod_spec_df_55['접합강도_y'] = prod_spec_df_55['접합강도_y'].astype(float)
prod_spec_df_55['인발력_상태'] = prod_spec_df_55['인발력_상태'].astype(float)
prod_spec_df_55['인발력_상온 침수 후(23°C, 72 h)'] = prod_spec_df_55['인발력_상온 침수 후(23°C, 72 h)'].astype(float)
prod_spec_df_55['마모강도(2000회)'] = prod_spec_df_55['마모강도(2000회)'].astype(float)
prod_spec_df_55['방염성능(45°법)_잔염시간'] = prod_spec_df_55['방염성능(45°법)_잔염시간'].astype(float)
prod_spec_df_55['방염성능(45°법)_탄화거리'] = prod_spec_df_55['방염성능(45°법)_탄화거리'].astype(float)
prod_spec_df_55['충격 흡수성'] = prod_spec_df_55['충격 흡수성'].astype(float)
prod_spec_df_55['수직 방향 변형'] = prod_spec_df_55['수직 방향 변형'].astype(float)
prod_spec_df_55['회전 저항'] = prod_spec_df_55['회전 저항'].astype(float)
prod_spec_df_55['피부 표면 마찰'] = prod_spec_df_55['피부 표면 마찰'].astype(float)
prod_spec_df_55['공의 반발력'] = prod_spec_df_55['공의 반발력'].astype(float)
prod_spec_df_55['스터드마모_충격 흡수성'] = prod_spec_df_55['스터드마모_충격 흡수성'].astype(float)
prod_spec_df_55['스터드마모_수직 방향 변형'] = prod_spec_df_55['스터드마모_수직 방향 변형'].astype(float)
prod_spec_df_55['스터드마모_파일 인발력'] = prod_spec_df_55['스터드마모_파일 인발력'].astype(float)
prod_spec_df_55['투수 성능'] = prod_spec_df_55['투수 성능'].astype(float)

In [269]:
prod_spec_df_55[comp_col]

,원사단사섬도,단위면적당 파일사 무게,파일사 무게 비율,접합강도_y,인발력_상태,"인발력_상온 침수 후(23°C, 72 h)",마모강도(2000회),방염성능(45°법)_잔염시간,방염성능(45°법)_탄화거리,충격 흡수성,수직 방향 변형,회전 저항,피부 표면 마찰,공의 반발력,스터드마모_충격 흡수성,스터드마모_수직 방향 변형,스터드마모_파일 인발력,투수 성능
0,2000.0,1650.0,0.0,250.0,80.0,80.0,10.0,20.0,10.0,50.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,180.0
1,2000.0,1650.0,0.0,250.0,80.0,80.0,10.0,20.0,10.0,50.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,180.0
2,2000.0,1650.0,0.0,250.0,80.0,80.0,10.0,20.0,10.0,50.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,180.0
3,2000.0,1650.0,0.0,250.0,80.0,80.0,10.0,20.0,10.0,50.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,180.0
4,2000.0,1650.0,0.0,250.0,80.0,80.0,10.0,20.0,10.0,50.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2000.0,2100.0,0.0,500.0,85.0,80.0,6.0,20.0,10.0,50.0,3.0,25.0,0.35,0.5,35.0,3.0,40.0,1000.0
417,2000.0,2100.0,0.0,500.0,85.0,80.0,6.0,20.0,10.0,50.0,3.0,25.0,0.35,0.5,35.0,3.0,40.0,1000.0
418,2000.0,1650.0,0.0,250.0,80.0,80.0,10.0,20.0,10.0,50.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,180.0
419,2000.0,2100.0,0.0,500.0,85.0,80.0,6.0,20.0,10.0,50.0,3.0,25.0,0.35,0.5,35.0,3.0,40.0,1000.0


In [272]:
prod_spec_df_55[prod_spec_df_55['우수조달물품여부']=='Y'][comp_col].mean()

원사단사섬도                     1976.213592
단위면적당 파일사 무게               1706.601942
파일사 무게 비율                     8.025890
접합강도_y                      515.695793
인발력_상태                       86.488673
인발력_상온 침수 후(23°C, 72 h)      81.294498
마모강도(2000회)                   5.983819
방염성능(45°법)_잔염시간              15.294498
방염성능(45°법)_탄화거리               8.401294
충격 흡수성                       50.110032
수직 방향 변형                      3.093851
회전 저항                        22.355987
피부 표면 마찰                      0.345146
공의 반발력                        0.484013
스터드마모_충격 흡수성                 25.469256
스터드마모_수직 방향 변형                2.148867
스터드마모_파일 인발력                 36.310680
투수 성능                       872.653722
dtype: float64